In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB

sys.path.insert(0, '../python/')
from helpers import *

plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 14})
warnings.filterwarnings('ignore')

In [ ]:
# features
# - hotel: [Resort Hotel, City Hotel] => [0, 1]
# - market_segment: [Corporate, Online TA, Offline TA/TO, Direct, Groups] => dummies
# - total_special_requests: [0, 1, 2] => dummies
# - room_difference: [0, 1]
# - party_size


In [13]:
data = pd.read_csv('../../data/train/cancellations.csv')

In [15]:
X = data[['hotel','market_segment', 'total_of_special_requests', 'total_nights', 'room_difference']].copy()
X.replace(['Resort Hotel', 'City Hotel'], [0, 1], inplace=True)
X.loc[np.argwhere((X['total_nights'] >= 8).values).flatten(), 'total_nights'] = 8
X.loc[np.argwhere((X['total_of_special_requests'] >= 2).values).flatten(), 'total_of_special_requests'] = 2
X.loc[np.argwhere(X['market_segment'].isin(['Aviation', 'Complementary']).values).flatten(), 'market_segment'] = 'Corporate'
#X = pd.get_dummies(X, columns=['market_segment', 'total_of_special_requests', 'total_nights']).to_numpy()
#y = data[['is_canceled']].to_numpy()

In [25]:
data['party_size'].value_counts()

2.0     65516
1.0     17783
3.0      8408
4.0      3160
0.0       138
5.0       101
26.0        5
27.0        2
20.0        1
12.0        1
10.0        1
50.0        1
40.0        1
Name: party_size, dtype: int64

In [27]:
data['total_nights'].value_counts()

2     21987
3     21718
1     16578
4     13852
7      6934
5      6249
6      3053
8       927
10      904
14      733
9       669
0       564
11      313
12      182
13      113
15       59
21       56
16       30
28       29
25       28
18       28
17       20
19       15
20       12
30       12
22       10
23        7
29        7
35        5
24        4
26        4
42        4
27        3
33        2
45        1
49        1
48        1
56        1
43        1
34        1
57        1
Name: total_nights, dtype: int64

In [31]:
data.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'total_nights',
       'party_size', 'is_family', 'room_difference', 'had_agent',
       'had_company', 'full_date'],
      dtype='object')